In [1]:
import pandas as pd
df = pd.read_pickle('data/df_encoded.pkl')
df.head()

,PT08.S1(CO),PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,is_weekend,month_April,...,month_November,month_October,month_September,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
datetime,,,,,,,,,,,,,,,,,,,,,
2004-10-03 18:00:00,1360.0,1046.0,1056.0,1692.0,1268.0,13.6,48.9,0.7578,1,0,...,0,1,0,0,0,0,1,0,0,0
2004-10-03 19:00:00,1292.0,955.0,1174.0,1559.0,972.0,13.3,47.7,0.7255,1,0,...,0,1,0,0,0,0,1,0,0,0
2004-10-03 20:00:00,1402.0,939.0,1140.0,1555.0,1074.0,11.9,54.0,0.7502,1,0,...,0,1,0,0,0,0,1,0,0,0
2004-10-03 21:00:00,1376.0,948.0,1092.0,1584.0,1203.0,11.0,60.0,0.7867,1,0,...,0,1,0,0,0,0,1,0,0,0
2004-10-03 22:00:00,1272.0,836.0,1205.0,1490.0,1110.0,11.2,59.6,0.7888,1,0,...,0,1,0,0,0,0,1,0,0,0
